In [8]:
# importing dependencies
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [9]:
# Import dotenv values
load_dotenv(".env")
DB_USER = os.getenv("DB_USER")
DB_PASSWD = os.getenv("DB_PASSWD")

In [10]:
#importing imdb files
imdb_ratings_path = "resources/imdb_ratings.csv"
ratings_df = pd.read_csv(imdb_ratings_path)


In [11]:
ratings_df.columns
ratings_min = ratings_df[['imdb_title_id','us_voters_rating', 'non_us_voters_rating','us_voters_votes', 'non_us_voters_votes']]
ratings_min.head()

,imdb_title_id,us_voters_rating,non_us_voters_rating,us_voters_votes,non_us_voters_votes
0,tt0000009,6.4,6.0,51.0,70.0
1,tt0000574,6.0,6.2,96.0,331.0
2,tt0001892,6.2,5.9,31.0,123.0
3,tt0002101,5.5,4.7,207.0,105.0
4,tt0002130,7.0,7.0,488.0,1166.0


In [12]:
cols = ratings_min.loc[:,'us_voters_rating':'non_us_voters_rating']
#ratings_min['average_total_ratings'] = cols.mean(axis=1)
#ratings_min.loc[:,'average_total_ratings'] = cols.mean(axis=1)

# v0.16.0 and later use the 'assign()' method
ratings_min = ratings_min.assign(avg_total_ratings=ratings_min.loc[:, ["us_voters_rating", "non_us_voters_rating"]].mean(axis=1))
ratings_min.head()

,imdb_title_id,us_voters_rating,non_us_voters_rating,us_voters_votes,non_us_voters_votes,avg_total_ratings
0,tt0000009,6.4,6.0,51.0,70.0,6.20
1,tt0000574,6.0,6.2,96.0,331.0,6.10
2,tt0001892,6.2,5.9,31.0,123.0,6.05
3,tt0002101,5.5,4.7,207.0,105.0,5.10
4,tt0002130,7.0,7.0,488.0,1166.0,7.00


In [13]:
col1 = ratings_min.loc[:,'us_voters_votes':'non_us_voters_votes']

ratings_min['average_total_votes'] = col1.mean(axis=1)
ratings_min.head()

,imdb_title_id,us_voters_rating,non_us_voters_rating,us_voters_votes,non_us_voters_votes,avg_total_ratings,average_total_votes
0,tt0000009,6.4,6.0,51.0,70.0,6.20,60.5
1,tt0000574,6.0,6.2,96.0,331.0,6.10,213.5
2,tt0001892,6.2,5.9,31.0,123.0,6.05,77.0
3,tt0002101,5.5,4.7,207.0,105.0,5.10,156.0
4,tt0002130,7.0,7.0,488.0,1166.0,7.00,827.0


In [14]:
connection_string = f"{DB_USER}:{DB_PASSWD}@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.connect()

# Insert data into database table
ratings_min.to_sql(name='ratings', con=engine, if_exists='replace', index=False)

# Close and Dispose of DB connection
conn.close()
engine.dispose()